In [1]:
import numpy as np
import pandas as pd
import os
import zipfile
import cv2
from matplotlib import pyplot as plt
import shutil
from tqdm import tqdm
import torch
import torchvision
import time
import copy
from torchvision import transforms, models

In [4]:
data_root = ".\\dataset"
print(os.listdir(data_root))

['test', 'train', 'val']


In [29]:
train_dir = os.path.join(data_root, "train")
print(train_dir)
class_names = ["is_human", "no"]

.\dataset\train


In [23]:
# train_transforms = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor()
# ])

# train_dataset = torchvision.datasets.ImageFolder(train_dir, train_transforms)

# print(train_dataset)

# batch_size = 1
# train_dataloader = torch.utils.data.DataLoader(
#     train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

Dataset ImageFolder
    Number of datapoints: 2
    Root location: .\dataset\train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )


In [24]:
def show_input(input_tensor, title='', num_batch=0, num_elem_batch=0):
    image = input_tensor.permute(1, 2, 0).numpy()
    plt.imshow(image.clip(0, 1))
    plt.title(str(num_batch) + "." + str(num_elem_batch) + ") " + title)
    plt.show()
    plt.pause(0.001)

def show_input_all(dataloader):
    it = iter(dataloader)
    for i in range(len(dataloader)):
        X_batch, y_batch = next(it)
        num_elem_batch = 0
        for x_item, y_item in zip(X_batch, y_batch):
            show_input(x_item, title=class_names[y_item], num_batch=i+1, num_elem_batch=num_elem_batch+1)
            num_elem_batch += 1

In [ ]:
# show_input_all(train_dataloader)

In [26]:
def save_input(input_tensor, title='', num_batch=0, num_elem_batch=0):
    image = input_tensor.permute(1, 2, 0).numpy()
    plt.imshow(image.clip(0, 1))
    plt.savefig(os.path.join(train_dir, title, "ch_0.jpg"))
    # plt.title(str(num_batch) + "." + str(num_elem_batch) + ") " + title)
    # plt.show()
    plt.pause(0.001)

def save_input_all(dataloader):
    it = iter(dataloader)
    for i in range(len(dataloader)):
        X_batch, y_batch = next(it)
        num_elem_batch = 0
        for x_item, y_item in zip(X_batch, y_batch):
            save_input(x_item, title=class_names[y_item], num_batch=i+1, num_elem_batch=num_elem_batch+1)
            num_elem_batch += 1

In [ ]:
# save_input_all(train_dataloader)

In [ ]:
# f_cascade = cv2.CascadeClassifier("H:/haarcascade_frontalface_default.xml")
# for imageName in glob.glob('H:/Для теста/*.jpg'):
#     img = (Image(filename=imageName))
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     faces = f_cascade.detectMultiScale(gray)
#     for x, y, width, height in faces:
#         cv2.rectangle(img, (x, y), (x + width, y + height), color=(255, 0, 0), thickness=2)   
#     cv2.imwrite("H:/Для теста/Тестированное/*.jpg", img)
#     outfile = path.join("H:/Для теста/Тестированное/", path.split(imageName)[1])
#     cv2.imwrite(outfile, img)

In [40]:
for class_name in class_names:
    source_dir = os.path.join(train_dir, class_name)
    for file_name in tqdm(os.listdir(source_dir)):
        img = cv2.imread(os.path.join(source_dir, file_name), cv2.IMREAD_COLOR)
        # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224))
        # for x, y, width, height in faces:
        #     cv2.rectangle(img, (x, y), (x + width, y + height), color=(255, 0, 0), thickness=2)   
        cv2.imwrite(os.path.join(source_dir, "resized_224_" + file_name), img)

100%|██████████| 1/1 [00:00<00:00, 151.87it/s]
